<a href="https://colab.research.google.com/github/anaperez10/Analiticalll-Finanzas/blob/main/Analitica_3_Seguros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Carga de librerias, paquetes y bases de datos**

In [ ]:
#Conexión con colab
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from unicodedata import normalize

In [ ]:
#Importar paquetes
import pandas as pd
import numpy as np
from unicodedata import normalize
from sklearn.model_selection import RandomizedSearchCV
import plotly.graph_objs as go #Graficos
import plotly.express as px
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt ### gráficos
from sklearn import linear_model ## para regresión lineal
from sklearn import tree ###para ajustar arboles de decisión
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor ##Ensamble con bagging
from sklearn.ensemble import GradientBoostingRegressor ###Ensamble boosting
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

In [ ]:
#Leer cada una de las bases de datos
expustos= pd.read_table('/content/drive/MyDrive/CursoAnalitica3/BD_Expuestos.txt')
socio =pd.read_table('/content/drive/MyDrive/CursoAnalitica3/BD_Sociodemograficas.txt', encoding='latin-1')
siniestros =pd.read_table('/content/drive/MyDrive/CursoAnalitica3/DB_Siniestros.txt',encoding='latin-1')

**Limpieza y transformación de datos**

Base de datos de expuestos

In [ ]:
#Observar los primeros cinco datos de la base de datos expuestos 
expustos.head(5)

In [ ]:
#Pasar el nombre de las columnas a minusculas
print(expustos.columns)
expustos.columns= expustos.columns.str.lower()

In [ ]:
#Tipo de variables de la base de datos expuestos 
expustos.dtypes

In [ ]:
#Dimensión de base de datos expuestos 
expustos.shape

In [ ]:
#Información de base de datos expuestos 
#No hay presencia de datos nulos
expustos.info()

In [ ]:
#Cambar el tipo de dato de las fechas a datetime
expustos['fecha_inicio']=pd.to_datetime(expustos['fecha_inicio'])
expustos['fecha_cancelacion']=pd.to_datetime(expustos['fecha_cancelacion'])
expustos['fecha_fin']=pd.to_datetime(expustos['fecha_fin'])

In [ ]:
#Verificar el tipo de datos de las fechas
expustos.dtypes

In [ ]:
#Reemplazar datos nulos por 0
expustos=expustos.fillna(0)

In [ ]:
#Función de categorizar la variable fecha de cancelación en 0 y 1
def categorizar(fila):
    fecha_cancelacion = fila.fecha_cancelacion
    if fecha_cancelacion  == 0:
      return 0
    else:
      return 1


In [ ]:
#Crear nuevo campo llamado Cancelación con la función categorizar
expustos["Cancelacion"] = expustos.apply(categorizar, axis=1)

In [ ]:
#Observar nuevamente la base de datos de expuestos al realizar la depuración de los datos
expustos

Base de datos de siniestros

In [ ]:
#Observar los primeros cinco datos de la base de datos de siniestros
siniestros.head(5)

In [ ]:
#Observar columnas de base de datos siniestros y pasar columnas a minusculas
siniestros.columns
siniestros.columns= siniestros.columns.str.lower()

In [ ]:
#Tipo de datos de la base de datos siniestros
siniestros.dtypes

In [ ]:
#Dimensiones de la base de datos siniestros
siniestros.shape

In [ ]:
#Información de la base de datos siniestros 
siniestros.info()

In [ ]:
#Observar si existen datos nulos 
siniestros.isnull().sum()

In [ ]:
#Imprimir categorias de las variables
lista = ['reclamacion','diagnostico_codigo','diagnostico_desc']
for i in lista:
    print(siniestros[i].value_counts())
    print('------------------------------------')

Base de datos sociodemograficas

In [ ]:
#Observar los primeros cinco datos de la base de datos sociodemografica
socio.head(5)

In [ ]:
#Observar columnas de la base de datos sociodemografica y pasar las columnas a minusculas
socio.columns
socio.columns= socio.columns.str.lower()

In [ ]:
#Tipos de datos de la base de datos sociodemografica
socio.dtypes

In [ ]:
#Convertir tipo de dato de object a datetime de la fecha de nacimiento
socio['fechanacimiento']=pd.to_datetime(socio['fechanacimiento'])

In [ ]:
#Observar dimensiones de la base de datos sociodemografica 
socio.shape

In [ ]:
#Información de la base de datos sociodemografica
socio.info()

In [ ]:
#Imprimir categorias de las variables
lista = ['sexo_cd','regional','cancer',
       'epoc', 'diabetes', 'hipertension', 'enf_cardiovascular']
for i in lista:
    print(socio[i].value_counts())
    print('------------------------------------')

In [ ]:
#Pasar los nulos de usuarios a sin información
socio['sexo_cd'] = socio['sexo_cd'].replace('-1','F')

In [ ]:
#Eliminar el resto de datos nulos
socio=socio.dropna()

In [ ]:
#Calcular edad 
socio['edad']=2019-socio.fechanacimiento.dt.year

In [ ]:
#Contar los valores de edad
socio['edad'].value_counts()

In [ ]:
#Volver la edad 0 si la resta es un valor menor a cero
socio.loc[socio[socio.edad<0].index,['edad']]=0

In [ ]:
#Función para categorizar la edad 
def categorizar1(fila):
    edad = fila.edad
    if edad  < 6:
      return 'Primera infancia'
    elif edad < 11:
      return 'Infancia'
    elif edad < 18:
      return 'Adolescencia'
    elif edad < 28:
      return 'Jovenes'
    elif edad <59:
      return'Adultez'
    elif edad >60:
      return 'Persona Mayor'


In [ ]:
#Crear variable curso de vida donde se aplica la función categorizar1 para clasificar los datos 
socio["curso de vida"] = socio.apply(categorizar1, axis=1)

Union de las bases de datos

In [ ]:
#Se une la base de datos sociodemograficas y la base de datos de siniestros 
dfcx = pd.merge(socio,siniestros, on = 'asegurado_id',how = 'inner')
dfcx.head(3)

In [ ]:
#Se observa si existe algún dato nulo
dfcx.isnull().sum()

In [ ]:
#Se une la base de datos creada anteriormente dfcx con expuestos1
dfcx1 = pd.merge(dfcx,expustos, on = 'asegurado_id',how = 'inner')
dfcx1.head(3)

In [ ]:
#Observar dimensión de nueva base de datos 
dfcx1.shape

In [ ]:
#Observar si existe la presencia de datos nulos
dfcx1.isnull().sum()

In [ ]:
#Se elimina la variable de curso de vida
dfcx1=dfcx1.drop(['curso de vida'], axis=1)

In [ ]:
#Se crea nuevamente el campo de curso de vida y se aplica la función de categorizar
dfcx1["curso de vida"] = dfcx1.apply(categorizar1, axis=1)

In [ ]:
#Contar los valores del campo curso de vida
dfcx1['curso de vida'].value_counts()

In [ ]:
#Eliminar datos nulos de la base de datos unida 
dfcx1=dfcx1.dropna()

**Análisis Exploratorio**

In [ ]:
df=pd.Series(expustos.fecha_fin - expustos.fecha_inicio).dt.days.value_counts().reset_index().rename(columns={'index':'Dias',0:'tamaño'})

In [ ]:
#Tiempo en dias en que las personas piden los seguros.
fig=px.bar(df.drop(index=[0,1]),x='Dias', y='tamaño', color_discrete_sequence =  px.colors.sequential.Aggrnyl)
fig.update_layout(font=dict(size=15)) 
fig.show()

In [ ]:
#Distribucion de las personas que cancenlan el seguro antes de terminar
# crear dataset
dic = {0:'No canceló',
       1:'Canceló'}
base = expustos.groupby(['Cancelacion'])[['asegurado_id']].count().sort_values('asegurado_id', ascending = False).reset_index()
base['Cancelacion'] = base['Cancelacion'].replace(dic)
# crear gráfica
fig = px.pie(base, values = 'asegurado_id', names ='Cancelacion',
             title= '<b>Participación de las personas que cancenlan el seguro antes de terminar<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [ ]:
df=pd.Series(expustos.fecha_inicio.dt.month_name().value_counts()).reset_index().rename(columns={'index':'Mes','fecha_inicio':'tamaño'})

In [ ]:
#Diagrama de barras de mes y tamaño
fig=px.bar(df,x='Mes',y='tamaño',color_discrete_sequence =  px.colors.sequential.Aggrnyl )
fig.update_layout(font=dict(size=15))
fig.show()

Exploración de base de datos Siniestros

In [ ]:
# crear dataset
base = siniestros.groupby(['diagnostico_desc'])[['valor_pagado']].sum().sort_values('valor_pagado', ascending = False).reset_index().head(10)

# crear gráfica
fig = px.bar(base, x = 'diagnostico_desc', y='valor_pagado',
             title= '<b>Costos por el diagnostico<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [ ]:
# crear dataset
base = siniestros.groupby(['reclamacion'])[['valor_pagado']].sum().sort_values('valor_pagado', ascending = False).reset_index().head(10)

# crear gráfica
fig = px.bar(base, x = 'reclamacion', y='valor_pagado',
             title= '<b>Costos por el tipo de reclamacion<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

Exploracion de base de datos Socio 

In [ ]:
# crear dataset
base = socio.groupby(['sexo_cd'])[['asegurado_id']].count().sort_values('asegurado_id', ascending = False).reset_index()

# crear gráfica
fig = px.pie(base, values = 'asegurado_id', names ='sexo_cd',
             title= '<b>Participación del sexo en los seguros de vida b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [ ]:
# crear dataset
base = socio.groupby(['curso de vida'])[['asegurado_id']].count().sort_values('asegurado_id', ascending = False).reset_index()

# crear gráfica
fig = px.pie(base, values = 'asegurado_id', names ='curso de vida',
             title= '<b>Participación del curso de vida en los seguros de vida b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [ ]:
# crear dataset
base = socio.groupby(['regional'])[['asegurado_id']].count().sort_values('asegurado_id', ascending = False).reset_index().head(10)

# crear gráfica
fig = px.bar(base, x = 'regional', y='asegurado_id',
             title= '<b>Cantidad de personas por region<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

Análisis exploratorio de base de datos concatenada

In [ ]:
#Columnas de la base de datos unida
dfcx1.columns

In [ ]:
#Sacar el año de la fecha de inicio
dfcx1['año'] = dfcx1['fecha_inicio'].dt.year 

In [ ]:
# crear dataset

base = dfcx1.groupby(['curso de vida', 'año'])[['valor_pagado']].sum().reset_index()
# MSSubClass: Tipo de edificio
# YrSold: Año en que se vende
# SalePrice: Precio de venta

# crear gráfica
fig = px.bar(base, x = 'año', y='valor_pagado', color = 'curso de vida', barmode = 'group', title= '<b>costos según el curso de vida por año<b>')

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [ ]:
# crear dataset
base = dfcx1.groupby(['sexo_cd', 'año'])[['valor_pagado']].sum().reset_index()

# crear gráfica
fig = px.bar(base, x = 'año', y='valor_pagado', color = 'sexo_cd', barmode = 'group', 
             title= '<b>Precio de venta según el tipo de acceso al edificio<b>',
             color_discrete_sequence=px.colors.qualitative.Antique)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

**Selección de variables**

In [ ]:
#Convertir las columnas de variables en minusculas 
dfcx1.columns = dfcx1.columns.str.lower()
#Inforación general de la base de datos
dfcx1.info()

In [ ]:
#Imprimir categorias de las variables
lista = ['sexo_cd', 'regional', 'cancer',
       'epoc', 'diabetes', 'hipertension', 'enf_cardiovascular','mes_pago', 
       'reclamacion', 'diagnostico_codigo', 'diagnostico_desc',
       'poliza_asegurado_id','cancelacion', 'curso de vida','año']
for i in lista:
    print(dfcx1[i].value_counts())
    print('------------------------------------')

In [ ]:
# Aplicar funciones para homologar a nivel general.
lista = ['sexo_cd', 'regional' ,'reclamacion', 'diagnostico_desc']
for i in lista:
    dfcx1[i] = dfcx1[i].str.lower() #minuscula
    dfcx1[i] = dfcx1[i].str.strip() #espacios
    dfcx1[i] = dfcx1[i].apply(lambda x: normalize("NFKD", x).encode("ascii", "ignore").decode("ascii") if pd.notnull(x) else x)

In [ ]:
#Observar los primeros cinco datos de la base de datos despúes de la depuración
dfcx1.head(5)

In [ ]:
#Eliminar datos duplicados de la base de datos 
dfcx1 = dfcx1.drop_duplicates()
#Observar la dimensión de la base de datos despúes de eliminar duplicados 
dfcx1.shape

In [ ]:
#Contar los valores de diagnostico_desc
df1 = pd.DataFrame(dfcx1['diagnostico_desc'].value_counts().reset_index())
df1

In [ ]:
#Eliminar datos que sean menores de 100 con el fin que esto no aporta con la predicción de la variable target
df2 = df1.drop(df1[df1['diagnostico_desc']<100].index)
df2

In [ ]:
#Renombras las columnas de df2
df2= df2.rename(columns={'index':'diagnostico_desc','diagnostico_desc':'cuenta'})

In [ ]:
#Observar df2 despúes de los cambios de nombre que se realizaron
df2

In [ ]:
#Unir bases de datos df2 y dfcx1
dfcx3 = pd.merge(df2,dfcx1, on = 'diagnostico_desc',how = 'inner')
dfcx3.head(3)

In [ ]:
#Observar dimensiones de la nueva base de datos
dfcx3.shape

In [ ]:
#Eliminar datos duplicados y seleccionando el último dato de asegurad_id
dfcx3=dfcx3.drop_duplicates(subset=['asegurado_id'])

In [ ]:
#Observar si existe la presencia de datos nulos 
dfcx3.isnull().sum()

In [ ]:
#Contar datos del campo diagnostico_desc
dfcx3['diagnostico_desc'].value_counts()

In [ ]:
#Eliminar los siguientes campos de la base de datos dfcx3
dfcx3 = dfcx3.drop(['asegurado_id', 'fechanacimiento',
       'mes_pago', 'diagnostico_codigo','poliza_asegurado_id', 'fecha_inicio',
       'fecha_cancelacion', 'fecha_fin', 'cancelacion','curso de vida','cuenta'],axis=1)

In [ ]:
# Convertir variableas categóricas en dummies
dfcx3_dummies = pd.get_dummies(dfcx3,columns=['sexo_cd','regional','reclamacion','diagnostico_desc'])
dfcx3_dummies.shape

In [ ]:
#Observar las dimensiones de la base datos depurada
dfcx3_dummies.shape

In [ ]:
# Descargar la BD depurada
dfcx3_dummies.to_csv('nueva.csv')

In [ ]:
#Determinar las columna target
X = dfcx3_dummies.loc[:,~dfcx3_dummies.columns.isin(['valor_pagado'])]
y = dfcx3_dummies['valor_pagado']

**Selección del modelo**

In [ ]:
# Modelos candidatos, teniendo en cuenta que es un problema de regresión
m_lreg = linear_model.LinearRegression()
m_rtree=tree.DecisionTreeRegressor()
m_rf= RandomForestRegressor()
m_gbt=GradientBoostingRegressor()

In [ ]:
# Se crea una lista con los anteriores modelos
modelos=list([m_lreg,m_rtree, m_rf, m_gbt])

In [ ]:
#Se crea una función para determinar que variables se van a elegir, teniendo en cuenta la lista anteriormente mencionada
def sel_variables(modelos,X,y, SelectFromModel,np,threshold):
    
    var_names_ac=np.array([])
    for modelo in modelos:
        #modelo=modelos[i]
        modelo.fit(X,y)
        sel = SelectFromModel(modelo, prefit=True,threshold=threshold)
        var_names= sel.get_feature_names_out(modelo.feature_names_in_)
        var_names_ac=np.append(var_names_ac, var_names)
        var_names_ac=np.unique(var_names_ac)
    
    return var_names_ac

In [ ]:
# Variables seleccionadas, en este caso se trabajarán con 118
var_names = sel_variables(modelos, X, y, SelectFromModel, np, threshold="3*mean")
var_names.shape

In [ ]:
# Nombres de las variables seleccionadas
var_names

In [ ]:
# Matriz con variables seleccionadas
X2 = X[var_names]

In [ ]:
# Se crea una función para poder elegir el mejor modelo de los 4 anteriormente mencionados.
def medir_modelos(modelos,scoring,X,y,cv,cross_val_score,pd):

    metric_modelos=pd.DataFrame()
    for modelo in modelos:
        scores=cross_val_score(modelo,X,y, scoring=scoring, cv=cv )
        pdscores=pd.DataFrame(scores)
        metric_modelos=pd.concat([metric_modelos,pdscores],axis=1)
    
    metric_modelos.columns=["reg_lineal","decision_tree","random_forest","gradient_boosting"]
    return metric_modelos

In [ ]:
# Se analiza el mape y rmse para definir el mejor modelo.
mape_varsel= medir_modelos(modelos,"neg_mean_absolute_percentage_error",X2,y,10, cross_val_score, pd)
rmse_varsel= medir_modelos(modelos,"neg_root_mean_squared_error",X2,y,10, cross_val_score, pd)

In [ ]:
#Mape de los diferentes modelos
mape_varsel.plot(kind='box')

**Afinamiento de hiperparámetros y evaluación del modelo**

In [ ]:
# Hiperparametros
param_grid = [{'n_estimators': [3, 500, 100], 'max_features': [5,20]},
{'bootstrap': [False,True]}]


tun_rf=RandomizedSearchCV(m_rf,param_distributions=param_grid,n_iter=3,scoring="neg_root_mean_squared_error")
tun_rf.fit(X2,y)

resultados=tun_rf.cv_results_
tun_rf.best_params_
pd_resultados=pd.DataFrame(resultados)
pd_resultados[["params","mean_test_score"]]

rf_final=tun_rf.best_estimator_ ### Guardar el modelo con hyperparameter tunning
m_lreg=m_lreg.fit(X2,y)

In [ ]:
#Observar los resultados 
pd_resultados[["params","mean_test_score"]]

In [ ]:
#Evaluar con cross validation
eval =cross_validate(rf_final,X2,y,cv=5,scoring="neg_root_mean_squared_error",return_train_score=True)

In [ ]:
train_rf=pd.DataFrame(eval['train_score'])
test_rf=pd.DataFrame(eval['test_score'])
train_test_rf=pd.concat([train_rf, test_rf],axis=1)
train_test_rf.columns=['train_score','test_score']

In [ ]:
#Promedio de test score
train_test_rf["test_score"].mean()

In [ ]:
#Vector de predicciones generadas 
predictions=cross_val_predict(m_lreg,X2,y,cv=5)
predictions

In [ ]:
#Histograma del error
pred=pd.DataFrame(predictions,columns=['pred'])
pdy=y.to_frame()
error=pdy['valor_pagado']-pred['pred']
error.hist(bins=50)

In [ ]:
#Boxplot del error
plt.boxplot(error,vert=False)
plt.show()

In [ ]:
#Evaluación de overfitting y underfitting
results = cross_validate(rf_final,X2,y,return_train_score=True,cv=5) 
results
test_scores = results['test_score']
train_scores = results['train_score']
#score de entrenamiento
print(np.mean(train_scores)) 
#score de prueba
print(np.mean(test_scores))

In [ ]:
# División de los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(
                                        X2,
                                        y,
                                        random_state = 123)

**Despliegue del modelo**